<a href="https://colab.research.google.com/github/charlotter62/EU-ETS-EUTL/blob/main/T3_transaction_xmls_byregistry_bydate_PARSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parse transaction XML files


---


**Description**:

The following code parses XML files containing transaction information, downloaded from the [European Union Transaction Log](https://ec.europa.eu/clima/ets/transaction.do). The files are downloaded and organized by registry and type of file with the [xml-byregistry-bydate.ipynb](https://colab.research.google.com/drive/1lmHfv5nGsRHqT0ce6R0OiZDq_JBmTrOe?usp=sharing) script.
* The DetailsAll.xml files are downloaded with the "DetailsAll" button at the bottom of the search results.
* The TransactionsBasic.xml files are downloaded with the "Export" button.

**Author**: Charlotte Rivard
**Contact**: 15crivard@gmail.com
**Date**: 1/13/2022

*Please reach out with questions and coauthorship considerations if using this script for publications*

---



In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install lxml
from lxml import objectify
import pandas as pd
import numpy as np
import os

In [ ]:
def parseDetailsAllXML(folder):
  all_transactions = pd.DataFrame();
  all_blocks = pd.DataFrame();

  if(os.path.exists(workingdir + folder +"/"+ folder.replace("/","_")+"_transactions.csv")):
    print("Details File already exists! " + folder.replace("/","_")+"_transactions.csv")
  else:
    print("Parsing... "+ folder.replace("/","_")+"_transactions.csv")
    files = [_ for _ in os.listdir(workingdir+folder) if _.endswith("DetailsAll.xml")] #os.listdir(path);

    for file in files:
      file = workingdir + folder + "/" + file
      xml_data = objectify.parse(file)  # Parse XML data
      transaction_tags = xml_data.findall("TransactionDetailsAll/Transaction")

      transactionsdf = pd.DataFrame();
      blocks = pd.DataFrame();
      blocknames = [];

      for tag in transaction_tags:
        row = tag.getchildren()
        aqflag = 0;
        transactionID = "";
        transactions = [];
        column_names = [];

        for item in row:
          if(len(item.getchildren())> 0): #If the list item has children, it is a transaction block
            blocknames = ["filename","transactionID"]+[b.tag for b in item.getchildren()]
            blockrow = [file,transactionID]+[b.text for b in item.getchildren()]
            blockrow = pd.DataFrame([blockrow],columns=blocknames)
            blocks = pd.concat([blocks,blockrow]).reset_index(drop=True) #blocks.append([blockrow])
          else:
            #colum_names.append(item.tag)
            if(item.tag=="AcquiringRegistry"):
                transactions.append(item.text)
                if(aqflag==0):
                  column_names.append(item.tag)
                  aqflag=1
                else:
                  column_names.append("AcquiringRegistryCode")
            else:
                if(item.tag not in column_names):
                  transactions.append(item.text)
                  column_names.append(item.tag)
                  if(item.tag=="TransactionID"):
                    transactionID = item.text;
        transactions = pd.DataFrame([transactions], columns=column_names)
        transactionsdf = pd.concat([transactionsdf,transactions]).reset_index(drop=True)

      #Save compiled datasets to csv
      all_transactions = pd.concat([all_transactions,transactionsdf]).reset_index(drop=True)
      savefile = workingdir + folder +"/"+ folder.replace("/","_")+"_transactions.csv"
      all_transactions.to_csv(savefile, index=False)

      all_blocks = pd.concat([all_blocks,blocks]).reset_index(drop=True)
      savefile = workingdir + folder +"/"+ folder.replace("/","_")+"_blocks.csv"
      all_blocks.to_csv(savefile, index=False)

  return([all_transactions, all_blocks])

In [ ]:
# workingdir = "/gdrive/MyDrive/Brookings/XML_downloads/xml-byregistry-bydate/"
# parseDetailsAllXML("BE/DetailsAll")

In [ ]:
def parseTransactionsXML(folder):
  savefile = workingdir + folder +"/"+ folder.replace("/","_")+"_transactions.csv"
  if(os.path.exists(savefile)):
    print("Basics File already exists! "+savefile)
    all_transactions = None
  else:
    print("Parsing... "+savefile)
    files = [_ for _ in os.listdir(workingdir+folder) if _.endswith("TransactionsBasic.xml")] #os.listdir(path);
    all_transactions = pd.DataFrame();

    for file in files:
      file = workingdir + folder + "/" + file
      xml_data = objectify.parse(file)  # Parse XML data
      transaction_tags = xml_data.findall("TransactionSearch/Transaction")

      transactionsdf = pd.DataFrame();

      for tag in transaction_tags:
        row = tag.getchildren()
        aqflag = 0;
        transactionID = "";
        transactions = [];
        column_names = [];

        for item in row:
          if(item.tag=="AcquiringRegistry"):
              transactions.append(item.text)
              if(aqflag==0):
                column_names.append(item.tag)
                aqflag=1
              else:
                column_names.append("AcquiringRegistryCode")
          else:
              if(item.tag not in column_names):
                transactions.append(item.text)
                column_names.append(item.tag)
                if(item.tag=="TransactionID"):
                  transactionID = item.text;
        transactions = pd.DataFrame([transactions], columns=column_names)
        transactionsdf = pd.concat([transactionsdf,transactions]).reset_index(drop=True)

      #Save compiled datasets to csv, saves the cumulative table each time a file is read in case the program fails. This could be adjusted if it slows the process too much.
      all_transactions = pd.concat([all_transactions,transactionsdf]).reset_index(drop=True)
      all_transactions.to_csv(savefile, index=False)

  return(all_transactions)

In [ ]:
parseTransactionsXML("BE/DetailsAll")

In loop

In [ ]:
import glob

In [ ]:
workingdir = "/gdrive/MyDrive/Brookings/XML_downloads/xml-byregistry-bydate/"
regfolders = glob.glob(workingdir+"*/")
regfolders

In [ ]:
# regfolders = [regfolders[3]]
# regfolders

['/gdrive/MyDrive/Brookings/XML_downloads/xml-byregistry-bydate/DK/']

In [ ]:
#for regfolder in regfolders[0:(len(regfolders)-1)]:
for regfolder in regfolders:

  regfolder = regfolder.replace(workingdir,"")

  #Transactions Basic
  regfolderTB = regfolder + "TransactionsBasic"
  # parsefilename = workingdir + regfolderTB +"/"+ regfolderTB.replace("/","_")+"_transactions.csv"
  # if(os.path.exists(parsefilename)):
  #   print("File already exists!\n" + parsefilename)
  # else:
  #   print("Parsing..." + parsefilename)
  parseTransactionsXML(regfolderTB)

  #Transactions Basic- Weekly, Daily
  if os.path.isdir(workingdir + regfolderTB +"/Monthly"):
    parseTransactionsXML(regfolderTB + "/Monthly")
  if os.path.isdir(workingdir + regfolderTB +"/Weekly"):
    parseTransactionsXML(regfolderTB + "/Weekly")
  if os.path.isdir(workingdir + regfolderTB +"/Daily"):
    parseTransactionsXML(regfolderTB + "/Daily")

  #Details All
  regfolderDA = regfolder + "DetailsAll"
  # parsefilename = workingdir + regfolderDA +"/"+ regfolderDA.replace("/","_")+"_transactions.csv"
  # if(os.path.exists(parsefilename)):
  #   print("File already exists!\n" + parsefilename)
  # else:
  #   print("Parsing..." + parsefilename)
  parseDetailsAllXML(regfolderDA)

  #Details All- Weekly, Monthly
  if os.path.isdir(workingdir + regfolderDA +"/Monthly"):
    parseDetailsAllXML(regfolderDA + "/Monthly")
  if os.path.isdir(workingdir + regfolderDA +"/Weekly"):
    parseDetailsAllXML(regfolderDA + "/Weekly")
  if os.path.isdir(workingdir + regfolderDA +"/Daily"):
    parseDetailsAllXML(regfolderDA + "/Daily")

#

# Combining csv files

https://pythonguides.com/python-get-all-files-in-directory/

In [ ]:
import glob
import pandas as pd

In [ ]:
workingdir = "/gdrive/MyDrive/Brookings/XML_downloads/xml-byregistry-bydate/"

DetailsAll_transactions

In [ ]:
# path = r"/gdrive/MyDrive/Brookings/XML_downloads/xml-byregistry-bydate/"
files = [f for f in glob.glob(workingdir + "**/*DetailsAll*transactions.csv", recursive=True)]
files

In [ ]:
# for f in files:
#   print(f)
#   print(f.split("/")[-1])
#   df = pd.read_csv(f,encoding='UTF-8')
#   print(df.shape)

In [ ]:
all_DetailsAll_transactions = pd.concat([pd.read_csv(f, encoding='UTF-8') for f in files])
all_DetailsAll_transactions = all_DetailsAll_transactions.drop_duplicates()
all_DetailsAll_transactions

In [ ]:
all_DetailsAll_transactions.to_csv(path+"all_DetailsAll_transactions.csv", index=False, encoding='utf-8-sig')

In [ ]:
# treg_counts = all_DetailsAll_transactions.groupby(['TransferringRegistry']).size()
# treg_counts
# treg_counts.to_csv("treg_counts.csv")

DetailsAll_blocks

In [ ]:
files = [f for f in glob.glob(workingdir + "**/*DetailsAll*blocks.csv", recursive=True)]
files

In [ ]:
all_DetailsAll_blocks = pd.concat([pd.read_csv(f, encoding='UTF-8') for f in files])
#all_DetailsAll_blocks = all_DetailsAll_blocks.drop_duplicates()
all_DetailsAll_blocks

In [ ]:
all_DetailsAll_blocks.to_csv(workingdir+"all_DetailsAll_blocks.csv", index=False, encoding='utf-8-sig')

TransactionsBasic_transactions

In [ ]:
files = [f for f in glob.glob(workingdir + "**/*TransactionsBasic*transactions.csv", recursive=True)]
files

In [ ]:
all_TransactionBasics_transactions = pd.concat([pd.read_csv(f, encoding='UTF-8') for f in files])
all_TransactionBasics_transactions = all_TransactionBasics_transactions.drop_duplicates()
all_TransactionBasics_transactions['TransactionType'] = all_TransactionBasics_transactions['TransactionType'].str.replace('-', '_')
all_TransactionBasics_transactions

In [ ]:
all_TransactionBasics_transactions = pd.read_csv(workingdir+"all_TransactionBasics_transactions.csv")

In [ ]:
all_TransactionBasics_transactions.to_csv(workingdir+"all_TransactionBasics_transactions.csv", index=False, encoding='utf-8-sig')

# Datetime experimentation

When you download the data as a csv and then open and save the csv in Excel, it drops part of the date time data. To avoid this, you could save the datetime variable as miliseconds, excel serial numbers, or a different standardized form. You could also split year,month,day, hours,minutes,seconds, microseconds, into their own columns.

In [ ]:
all_DetailsAll_transactions = pd.read_csv(workingdir+"all_DetailsAll_transactions.csv")

https://stackoverflow.com/questions/9574793/how-to-convert-a-python-datetime-datetime-to-excel-serial-date-number

In [ ]:
import datetime as dt
def excel_date(date1):
    temp = dt.datetime(1899, 12, 30)    # Note, not 31st Dec but 30th!
    delta = date1 - temp
    return float(delta.days) + (float(delta.seconds) / 86400.0)

In [ ]:
b = datetime(2005, 12, 28, 12, 3, 54, 581)
excel_date(b)

38714.50270833333

In [ ]:
b = datetime(2005, 12, 28, 12, 3, 55, 0)
excel_date(b)

38714.50271990741

In [ ]:
all_DetailsAll_transactions.to_excel(workingdir+"all_DetailsAll_transactions.xlsx", index=False)

https://pythoninoffice.com/save-data-to-excel-file-python/


In [ ]:
from datetime import datetime
def toDatetime(date_time_str):
  date = date_time_str.split(" ")[0]
  year = int(date.split("-")[0])
  month = int(date.split("-")[1])
  day = int(date.split("-")[2])

  time = date_time_str.split(" ")[1]
  hr = int(time.split(":")[0])
  min = int(time.split(":")[1])
  secs = time.split(":")[2]
  secs = format(float(secs),".3f")
  print(secs)
  sec = int(secs.split(".")[0])
  micro = int(secs.split(".")[1])

  d = datetime(year,month,day,hr,min,sec,micro)
  return excel_date(d)
  #
  #dt = date + " " + hrs + ":" + min + ":" + sec
  #return datetime.strptime(dt, '%Y-%m-%d %H:%M:%S.%f')

In [ ]:
toDatetime("2005-12-28 12:03:54.581")

54.581


38714.50270833333

In [ ]:
all_DetailsAll_transactions['TransactionDate'] = all_DetailsAll_transactions['TransactionDate'].apply(toDatetime)
all_DetailsAll_transactions

In [ ]:
all_DetailsAll_transactions.to_csv(workingdir+"all_DetailsAll_transactions2.csv",index=False, encoding='utf-8-sig')
#date_format='%Y-%m-%d %H:%M:%S',

# Archives

In [ ]:
def parseXMLfiles(folder):
  path = "./"#%pwd
  xmlfiles = [_ for _ in os.listdir(path +"/"+folder) if _.endswith("DetailsAll.xml")] #os.listdir(path);
  all_transactions = pd.DataFrame();
  all_blocks = pd.DataFrame();

  for file in xmlfiles:
    file = "./"+ folder + "/" + file
    xml_data = objectify.parse(file)  # Parse XML data
    transaction_tags = xml_data.findall("TransactionDetailsAll/Transaction")
    row = transaction_tags[0].getchildren()

    transactions = [];
    column_names = [];
    blocks = pd.DataFrame();
    blocknames = [];
    aqflag = 0;

    for item in row:
      if(len(item.getchildren())> 0): #If the list item has children, it is a transaction block
        blocknames = ["filename"]+[b.tag for b in item.getchildren()]
        blockrow = [file,transactionID]+[b.text for b in item.getchildren()]
        blockrow = pd.DataFrame([blockrow],columns=blocknames)
        blocks = pd.concat([blocks,blockrow]).reset_index(drop=True) #blocks.append([blockrow])
        #blockrow = [file] + [b.text for b in item.getchildren()]
        #print(blockrow)
        #blocks = blocks.append([blockrow])
      else:
        #colum_names.append(item.tag)
        if(item.tag=="AcquiringRegistry"):
            transactions.append(item.text)
            if(aqflag==0):
              column_names.append(item.tag)
              aqflag=1
            else:
              column_names.append("AcquiringRegistryCode")
        else:
            if(item.tag not in column_names):
              transactions.append(item.text)
              column_names.append(item.tag)
    transactions = pd.DataFrame([transactions], columns=column_names)
    #blocks.columns = blocknames
    #aligned = appendDiff(transactions, all_transactions)
    #transactions = aligned[0]
    #all_transactions = aligned[1]
    all_transactions = all_transactions.append(transactions)
    all_blocks = all_blocks.append(blocks)
    #all_blocks.columns = blocknames

  #print(blocknames)
  #After loops
  #print(outprefix + "_DetailsAll_xml_bycountry-year.csv")
  #all_transactions.to_csv(outprefix + "_DetailsAll_xml_bycountry-year.csv",encoding='utf-8')
  #print(outprefix + "_TransactionBlocks_xml_bytransactions.csv")
  #all_blocks.to_csv(outprefix + "_TransactionBlocks_xml_bytransactions.csv",encoding='utf-8')
  return([all_transactions, all_blocks])

In [ ]:
!ls

sample_data


In [ ]:
%cd /gdrive/MyDrive/Brookings/XML_downloads/xml-byregistry-bydate/

/gdrive/MyDrive/Brookings/XML_downloads/xml-byregistry-bydate


In [ ]:
parseXMLfiles("BE/DetailsAll")

NameError: ignored

In [ ]:
file = "./BE/DetailsAll/BE_2005-01-01_2006-01-01_DetailsAll.xml"
xml_data = objectify.parse(file)

In [ ]:
BE = parseXMLfiles("BE/DetailsAll")
BE_transactions = BE[0]
BE_blocks = BE[1]

['filename', 'OriginatingRegistry', 'NumberOfUnits', 'SuppUnitTypeCode', 'OriginalPeriodCode', 'ApplicablePeriodCode', 'UnitTypeCode']


In [ ]:
BE_blocks

In [ ]:
folder = "BE/DetailsAll"
file = "BE_2014-01-01_2015-01-01_DetailsAll.xml"
file = "./"+ folder + "/" + file
xml_data = objectify.parse(file)  # Parse XML data
transaction_tags = xml_data.findall("TransactionDetailsAll/Transaction")

transactionsall = pd.DataFrame();
blocks = pd.DataFrame();
blocknames = [];

for tag in transaction_tags:
  row = tag.getchildren()
  aqflag = 0;
  transactionID = "";
  transactions = [];
  column_names = [];

  for item in row:
    if(len(item.getchildren())> 0): #If the list item has children, it is a transaction block
      blocknames = ["filename","transactionID"]+[b.tag for b in item.getchildren()]
      blockrow = [file,transactionID]+[b.text for b in item.getchildren()]
      blockrow = pd.DataFrame([blockrow],columns=blocknames)
      blocks = pd.concat([blocks,blockrow]).reset_index(drop=True) #blocks.append([blockrow])
    else:
      #colum_names.append(item.tag)
      if(item.tag=="AcquiringRegistry"):
          transactions.append(item.text)
          if(aqflag==0):
            column_names.append(item.tag)
            aqflag=1
          else:
            column_names.append("AcquiringRegistryCode")
      else:
          if(item.tag not in column_names):
            transactions.append(item.text)
            column_names.append(item.tag)
            if(item.tag=="TransactionID"):
              transactionID = item.text;
  transactions = pd.DataFrame([transactions], columns=column_names)
  transactionsall = pd.concat([transactionsall,transactions]).reset_index(drop=True)

In [ ]:
transactionsall

,TransferringRegistry,TransferringAccountType,AcquiringRegistry,TransferringAccountTypeCode,AcquiringRegistryCode,AcquiringAccountHolderName,TransferringAccountHolderName,AcquiringAccountName,AcquiringAccountType,TransferringAccountName,TransactionDate,TransactionID,SuppTransactionTypeCode,TransactionTypeCodeLookup,TransactionType,SuppTransactionTypeCodeLookup
0,BE,Holding Account,BE,100,Belgium,Zandvliet Power,Electrabel,307 Zandvliet Power,100,089 Electrabel Turbo Jet back up Cierreux,2014-12-31 12:04:34.307,EU237235,0,Internal - Internal transfer of unit/supplemen...,10,no supp
1,BE,Holding Account,EU,100,European Commission,EUROPEAN COMMISSION,Raffinerie Tirlemontoise-Tiense Suikerraffinad...,EU International Credit Account,100,236 Tiense Suikerraffinaderij - vestiging Tienen,2014-12-29 10:59:31.064,EU227491,71,Internal - Internal transfer of unit/supplemen...,10,Exchange
2,BE,Holding Account,NL,100,Netherlands,ACT Financial Solutions B.V.,DUCATT,ACT Trading,100,396 DUCATT NV,2014-12-24 17:58:26.192,EU236974,0,Internal - Internal transfer of unit/supplemen...,10,no supp
3,BE,Holding Account,EU,100,European Commission,EUROPEAN COMMISSION,DUCATT,EU International Credit Account,100,396 DUCATT NV,2014-12-22 21:47:20.341,EU236658,71,Internal - Internal transfer of unit/supplemen...,10,Exchange
4,BE,Holding Account,BE,100,Belgium,Carmeuse,Carmeuse,902 Carmeuse SA,100,218 Carmeuse Four à chaux Seilles,2014-12-22 12:02:35.267,EU235007,0,Internal - Internal transfer of unit/supplemen...,10,no supp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,BE,Holding Account,NL,100,Netherlands,Electrabel NV/SA,Electrabel,Electrabel S.A./N.V.,100,089 Electrabel Turbo Jet back up Cierreux,2014-01-07 16:03:11.401,EU145639,0,Internal - Internal transfer of unit/supplemen...,10,no supp
953,BE,Holding Account,BE,100,Belgium,Electrabel,ArcelorMittal Belgium,074 Electrabel - Centrale Rodenhuize,100,011 ArcelorMittal Gent,2014-01-06 15:40:01.832,EU145592,0,Internal - Internal transfer of unit/supplemen...,10,no supp
954,BE,Holding Account,BE,100,Belgium,Carsid,Duferco Belgium,284 Carsid Aciérie Marcinelle,100,180 Duferco Belgium - Produits Longs La Louvière,2014-01-06 12:01:28.878,EU140762,0,Internal - Internal transfer of unit/supplemen...,10,no supp
955,BE,Holding Account,BE,100,Belgium,TotalEnergies Refinery Antwerp,TotalEnergies Olefins Antwerp,127 TotalEnergies Refinery Antwerp,100,222 TotalEnergies Olefins Antwerp,2014-01-03 17:52:48.864,EU145547,0,Internal - Internal transfer of unit/supplemen...,10,no supp


In [ ]:
blocks

,filename,transactionID,OriginatingRegistry,NumberOfUnits,OriginalPeriodCode,ApplicablePeriodCode,UnitTypeCode,SuppUnitTypeCode,ProjectID,Track
0,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU237235,BR,1095,1,1,5,0,181,NaN
1,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU237235,BR,87,1,1,5,0,1133,NaN
2,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU237235,BR,1362,1,1,5,0,181,NaN
3,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU237235,BR,1362,1,1,5,0,181,NaN
4,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU237235,CN,25934,1,1,5,0,5296,NaN
...,...,...,...,...,...,...,...,...,...,...
2004,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU145639,EU,27354,2,2,0,5,NaN,NaN
2005,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU145592,EU,396689,2,2,0,5,NaN,NaN
2006,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU140762,EU,110000,2,2,0,5,NaN,NaN
2007,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU145547,EU,200000,2,2,0,5,NaN,NaN


In [ ]:
transactions

In [ ]:
blocks.to_csv("blocks.csv")

In [ ]:
blocks = blocks.reset_index(drop=True)

In [ ]:
notrack = pd.DataFrame(blocks.loc[[0]])
notrack.columns = ["File","Transaction.ID","Registry.Code","Nb.of.Units","Project.ID","Supp.Unit.Type.Code","Original.Period.Code","Applicable.Period.Code","Unit.Type.Code","Extra"]
notrack = notrack.loc[0:,"File":"Unit.Type.Code"]
notrack

,File,Transaction.ID,Registry.Code,Nb.of.Units,Project.ID,Supp.Unit.Type.Code,Original.Period.Code,Applicable.Period.Code,Unit.Type.Code
0,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU237235,BR,1095,181,0,1,1,5


In [ ]:
track = pd.DataFrame(blocks.loc[[22]])
track.columns = ["File","Transaction.ID","Registry.Code","Nb.of.Units","Project.ID","Supp.Unit.Type.Code","Track","Original.Period.Code","Applicable.Period.Code","Unit.Type.Code"]
track

,File,Transaction.ID,Registry.Code,Nb.of.Units,Project.ID,Supp.Unit.Type.Code,Track,Original.Period.Code,Applicable.Period.Code,Unit.Type.Code
22,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU236658,DE,4413,1000197,0,1,1,1,3


https://www.statology.org/rbind-in-python/

In [ ]:
pd.concat([notrack,track]).reset_index(drop=True)

,File,Transaction.ID,Registry.Code,Nb.of.Units,Project.ID,Supp.Unit.Type.Code,Original.Period.Code,Applicable.Period.Code,Unit.Type.Code,Track
0,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU237235,BR,1095,181,0,1,1,5,NaN
1,./BE/DetailsAll/BE_2014-01-01_2015-01-01_Detai...,EU236658,DE,4413,1000197,0,1,1,3,1
